<a href="https://colab.research.google.com/github/lhenriquear/API_DOTNET_MONGO/blob/master/CounterStrikeAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## STARTS HERE
import pandas as pd
#merge economy and results.csv as df1
results = pd.read_csv('results.csv')
economy =pd.read_csv('economy.csv')
df1 = pd.merge(results, economy, on=['match_id'])

In [2]:
df1.head()

,date_x,team_1_x,team_2_x,_map_x,result_1,result_2,map_winner,starting_ct,ct_1,t_2,t_1,ct_2,event_id_x,match_id,rank_1,rank_2,map_wins_1,map_wins_2,match_winner,date_y,event_id_y,team_1_y,team_2_y,best_of,_map_y,t1_start,t2_start,1_t1,2_t1,3_t1,4_t1,5_t1,6_t1,7_t1,8_t1,9_t1,10_t1,11_t1,12_t1,13_t1,...,21_t2,22_t2,23_t2,24_t2,25_t2,26_t2,27_t2,28_t2,29_t2,30_t2,1_winner,2_winner,3_winner,4_winner,5_winner,6_winner,7_winner,8_winner,9_winner,10_winner,11_winner,12_winner,13_winner,14_winner,15_winner,16_winner,17_winner,18_winner,19_winner,20_winner,21_winner,22_winner,23_winner,24_winner,25_winner,26_winner,27_winner,28_winner,29_winner,30_winner
0,2020-03-01,Wings,Global,Dust2,16,13,1,1,7,8,9,5,5234,2339873,231,167,2,1,1,2020-03-01,5234,Wings,Global,3,Dust2,ct,t,4250.0,18950.0,14350.0,25300.0,3250.0,7650.0,24250.0,22950.0,8350.0,27750.0,25200.0,23700.0,28250.0,...,29050.0,17350.0,1000.0,23900.0,7500.0,28900.0,25400.0,26600.0,16500.0,NaN,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN
1,2020-03-01,Wings,Global,Dust2,16,13,1,1,7,8,9,5,5234,2339873,231,167,2,1,1,2020-03-01,5234,Wings,Global,3,Train,ct,t,4250.0,22450.0,21150.0,15250.0,8750.0,23100.0,13250.0,30850.0,30250.0,22250.0,2800.0,24950.0,11900.0,...,34500.0,33200.0,32100.0,32400.0,32800.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN
2,2020-03-01,Wings,Global,Dust2,16,13,1,1,7,8,9,5,5234,2339873,231,167,2,1,1,2020-03-01,5234,Wings,Global,3,Inferno,ct,t,4250.0,15850.0,20100.0,26700.0,31900.0,28850.0,27600.0,15950.0,1200.0,24350.0,27400.0,28600.0,28900.0,...,28550.0,29650.0,31750.0,30950.0,3500.0,21550.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,2.0,2.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
3,2020-03-01,Wings,Global,Inferno,16,10,1,1,9,6,7,4,5234,2339873,231,167,2,1,1,2020-03-01,5234,Wings,Global,3,Dust2,ct,t,4250.0,18950.0,14350.0,25300.0,3250.0,7650.0,24250.0,22950.0,8350.0,27750.0,25200.0,23700.0,28250.0,...,29050.0,17350.0,1000.0,23900.0,7500.0,28900.0,25400.0,26600.0,16500.0,NaN,1.0,2.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,1.0,2.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,NaN
4,2020-03-01,Wings,Global,Inferno,16,10,1,1,9,6,7,4,5234,2339873,231,167,2,1,1,2020-03-01,5234,Wings,Global,3,Train,ct,t,4250.0,22450.0,21150.0,15250.0,8750.0,23100.0,13250.0,30850.0,30250.0,22250.0,2800.0,24950.0,11900.0,...,34500.0,33200.0,32100.0,32400.0,32800.0,NaN,NaN,NaN,NaN,NaN,1.0,1.0,2.0,2.0,2.0,2.0,1.0,1.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN


In [3]:
#Defining the target and separating target from predictors
y = df1.match_winner
X = df1.drop(['match_winner'], axis = 1)

In [5]:
# Divide data into training and validation subsets
from sklearn.model_selection import train_test_split
X_train_full, X_valid_full, y_train,y_valid =train_test_split (X,y, train_size = 0.8, test_size = 0.2, random_state = 42) 

In [6]:
#SimpleInputer and Label Encoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

cols_with_missing = [col for col in X_train_full.columns if X_train_full[col].isnull().any()] 
X_train_full.drop(cols_with_missing, axis=1, inplace=True)
X_valid_full.drop(cols_with_missing, axis=1, inplace=True)

categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype == "object"]

numerical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

my_cols = categorical_cols + numerical_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

#Preprocessing steps
#Apply label enconder to each column with categorical data

numerical_transformer = SimpleImputer(strategy = 'constant')
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
preprocessor = ColumnTransformer(
    transformers = [
                    ('num', numerical_transformer, numerical_cols),
                    ('cat', categorical_transformer, categorical_cols)
                  ])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4174: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [7]:
#Defining model
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators=100, random_state=0)

my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

my_pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('num',
                                                  SimpleImputer(add_indicator=False,
                                                                copy=True,
                                                                fill_value=None,
                                                                missing_values=nan,
                                                                strategy='constant',
                                                                verbose=0),
                                                  ['result_1', 'result_2',
                                                   'map_winner', 'starting_ct',
                                                   'ct_1', 't_2

In [9]:
from sklearn.model_selection import cross_val_score

# Multiply by -1 since sklearn calculates *negative* MAE
scores = -1 * cross_val_score(my_pipeline, X, y,
                              cv=5,
                              scoring='neg_mean_absolute_error')

print("MAE scores:\n", scores)

MAE scores:
 [1.90249703e-05 1.98810939e-03 3.79548157e-03 3.32936980e-05
 4.75737393e-06]
